In [47]:
from datetime import datetime
import numpy as np
import pandas as pd
import struct

__version__ = '0.01.00'

# Constants
GDFNAMELEN = 16                  # Length of the ascii-names
GDFID = 94325877                # ID for GDF

# Data types
t_undef = int('0000', 16)       # Data type not defined
t_ascii = int('0001', 16)       # ASCII character
t_s32 = int('0002', 16)       # Signed long
t_dbl = int('0003', 16)       # Double
t_null = int('0010', 16)       # No data
t_u8 = int('0020', 16)       # Unsigned char
t_s8 = int('0030', 16)       # Signed char
t_u16 = int('0040', 16)       # Unsigned short
t_s16 = int('0050', 16)       # Signed short
t_u32 = int('0060', 16)       # Unsigned long
t_u64 = int('0070', 16)       # Unsigned 64bit int
t_s64 = int('0080', 16)       # Signed 64bit int
t_flt = int('0090', 16)       # Float

# Block types
t_dir = 256      # Directory entry start
t_edir = 512      # Directory entry end
t_sval = 1024      # Single valued
t_arr = 2048      # Array


# Top level gdf readers
def gdftomemory(gdf_file):
    all = []

    def myprocfunc(params, data):
        all.append({'p': params, 'd': data})

    print(readgdf(gdf_file, myprocfunc))
    return all


def gdftopandas(gdf_file):
    params = []
    data = []
    paramid = 1

    def myprocfunc(blockparams, blockdata):
        nonlocal paramid

        temp = blockparams.copy()
        temp['paramid'] = paramid
        params.append(temp)

        temp = blockdata.copy()
        try:
            templen = len(next(iter(temp.values())))
            tempcol = np.full((templen, ), paramid)
            temp['paramid'] = tempcol
        except StopIteration:
            pass
        data.append(temp)

        paramid += 1

    header = readgdf(gdf_file, myprocfunc)

    params = pd.DataFrame(params)
    params.set_index('paramid', inplace=True)

    data = pd.concat([pd.DataFrame(blockdata) for blockdata in data],
                     sort=False)
    data.reset_index(drop=True, inplace=True)

    return header, params, data


# Internal functions
def _parseblocks(f, params, procfunc):
    data = {}
    while True:
        # TODO check end of file?
        # Read block header
        name = f.read(GDFNAMELEN)
        if len(name) == 0:
            procfunc(params, data)
            return
        name = name.decode().rstrip('\x00')
        type_, = struct.unpack('i', f.read(4))
        size, = struct.unpack('i', f.read(4))

        # Get block type and data type
        is_dir = (type_ & t_dir != 0)
        is_edir = (type_ & t_edir != 0)
        is_sval = (type_ & t_sval != 0)
        is_arr = (type_ & t_arr != 0)

        dattype = type_ & 255

        # Get block data (scalar or array)
        if is_sval:
            if dattype == t_dbl:
                value = struct.unpack('d', f.read(8))[0]
            elif dattype == t_null:
                pass
            elif dattype == t_ascii:
                value = f.read(size).decode().rstrip('\x00')
            elif dattype == t_s32:
                value = struct.unpack('i', f.read(4))[0]
            else:
                print('unknown datatype of value!!!')
                print('name=', name)
                print('type=', type_)
                print('size=', size)
                print('dattype=', dattype)
                value = f.read(size)
        elif is_arr:
            if dattype == t_dbl:
                value = np.frombuffer(f.read(size))
                data[name] = value
            else:
                print('unknown datatype of value!!!')
                print('name=', name)
                print('type=', type_)
                print('size=', size)
                print('dattype=', dattype)
                value = f.read(size)

        # Take care of recursion
        if is_dir:
            myparams = params.copy()
            myparams[name] = value
            _parseblocks(f, myparams, procfunc)
        elif is_edir:
            procfunc(params, data)
            return
        elif is_sval:
            params[name] = value


def _proc_print(params, data):
    print(params)
    print(data.keys())
    print('_____________________')


# Base gdf reader
def readgdf(gdf_file, procfunc=_proc_print):
    # Output data
    gdf_head = {}

    with open(gdf_file, 'rb') as f:
        # Read the GDF main header
        gdf_id_check = struct.unpack('i', f.read(4))[0]
        if gdf_id_check != GDFID:
            raise RuntimeWarning('File is not a .gdf file')

        time_created = struct.unpack('i', f.read(4))[0]
        time_created = datetime.fromtimestamp(time_created)
        gdf_head['time_created'] = time_created.isoformat(' ')

        gdf_head['creator'] = f.read(GDFNAMELEN).decode().rstrip('\x00')

        gdf_head['destination'] = f.read(GDFNAMELEN).decode().rstrip('\x00')

        major = struct.unpack('B', f.read(1))[0]
        minor = struct.unpack('B', f.read(1))[0]
        gdf_head['gdf_version'] = str(major) + '.' + str(minor)

        major = struct.unpack('B', f.read(1))[0]
        minor = struct.unpack('B', f.read(1))[0]
        gdf_head['creator_version'] = str(major) + '.' + str(minor)

        major = struct.unpack('B', f.read(1))[0]
        minor = struct.unpack('B', f.read(1))[0]
        gdf_head['destination_version'] = str(major) + '.' + str(minor)

        f.seek(2, 1)   # skip 2 bytes to go to next block

        # Read GDF data blocks (starts recursive reading)
        _parseblocks(f, {}, procfunc)

    return gdf_head


In [48]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MaxNLocator, ScalarFormatter


data = gdftomemory("test.gdf")




def get_data(timestep, component):
    timestep_data = data[timestep].get("d")
    x = np.array(timestep_data.get("x"))
    y = np.array(timestep_data.get("y"))
    z = np.array(timestep_data.get("z"))
    q = np.array(timestep_data.get("q"))
    var = np.array(timestep_data.get(component))
    return x, y, z, q, var




def calculate_zoom_limits(x, y, z, zoom):
    x_range, y_range, z_range = x.max() - x.min(), y.max() - y.min(), z.max() - z.min()
    zoom_factor = (100 - zoom) / 100
    x_center, y_center, z_center = np.mean(x), np.mean(y), np.mean(z)
    x_lim = (x_center - x_range * zoom_factor / 2, x_center + x_range * zoom_factor / 2)
    y_lim = (y_center - y_range * zoom_factor / 2, y_center + y_range * zoom_factor / 2)
    z_lim = (z_center - z_range * zoom_factor / 2, z_center + z_range * zoom_factor / 2)
    return x_lim, y_lim, z_lim




def filter_data(x, y, z, q, var, x_lim, y_lim, z_lim):
    mask = (x >= x_lim[0]) & (x <= x_lim[1]) & (y >= y_lim[0]) & (y <= y_lim[1]) & (z >= z_lim[0]) & (z <= z_lim[1])
    return x[mask], y[mask], z[mask], q[mask], var[mask]




def plot_3d_scatter(ax, x, y, z, var, component, elevation, rotation, dummy, q):
    dummy_mask = q != 0
    if dummy:
        sc = ax.scatter(z, x, y, c=var, cmap='viridis', s=20, vmin=var.min(), vmax=var.max(), label=component)
    else:
        sc = ax.scatter(z[dummy_mask], x[dummy_mask], y[dummy_mask], c=var[dummy_mask], cmap='viridis', s=20, vmin=var[dummy_mask].min(), vmax=var[dummy_mask].max(), label=component)
    ax.view_init(elev=elevation, azim=rotation + 305)
    return sc




def plot_2d_slice(ax, x_slice, y_slice, var_slice, component, var, dummy, q_slice):
    dummy_slice_mask = q_slice != 0
    if dummy:
        sc = ax.scatter(x_slice, y_slice, c=var_slice, cmap='viridis', s=20, vmin=var.min(), vmax=var.max(), label=component)
    else:
        sc = ax.scatter(x_slice[dummy_slice_mask], y_slice[dummy_slice_mask], c=var_slice[dummy_slice_mask], cmap='viridis', s=20, vmin=var.min(), vmax=var.max(), label=component)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    return sc




def plot_original(timestep=0, component='q', zoom=60, slice_position=50, elevation=30, rotation=0, dummy=True):
    x, y, z, q, var = get_data(timestep, component)
    x_lim, y_lim, z_lim = calculate_zoom_limits(x, y, z, zoom)
    x_filtered, y_filtered, z_filtered, q_filtered, var_filtered = filter_data(x, y, z, q, var, x_lim, y_lim, z_lim)
    
    fig = plt.figure(figsize=(30, 13))

    ax1 = fig.add_subplot(121, projection='3d')
    sc1 = plot_3d_scatter(ax1, x_filtered, y_filtered, z_filtered, var_filtered, component, elevation, rotation, dummy, q_filtered)
    fig.colorbar(sc1, ax=ax1, label=component)
    ax1.set_xlim(z_lim)
    ax1.set_ylim(x_lim)
    ax1.set_zlim(y_lim)

    slice_z_value = z.min() + (slice_position / 100) * (z.max() - z.min())
    z_std = np.std(z_filtered)
    margin = max(0.03 * (z.max() - z.min()), z_std)
    slice_mask = (z_filtered > slice_z_value - margin) & (z_filtered < slice_z_value + margin)
    x_slice_zoomed, y_slice_zoomed, var_slice_zoomed, q_slice_zoomed = x_filtered[slice_mask], y_filtered[slice_mask], var_filtered[slice_mask], q_filtered[slice_mask]

    if x_slice_zoomed.size > 0 and y_slice_zoomed.size > 0 and var_slice_zoomed.size > 0:
        ax2 = fig.add_subplot(122)
        sc2 = plot_2d_slice(ax2, x_slice_zoomed, y_slice_zoomed, var_slice_zoomed, component, var_filtered, dummy, q_slice_zoomed)
        fig.colorbar(sc2, ax=ax2, label=component)
        ax2.set_title(f'Slice at z = {slice_z_value}')
    else:
        print("No data points found in the specified slice range.")

    plt.show()


{'time_created': '2024-08-05 13:04:40', 'creator': 'MR', 'destination': '', 'gdf_version': '1.1', 'creator_version': '1.3', 'destination_version': '0.0'}


In [49]:
import ipywidgets as widgets
from IPython.display import display

# Create the sliders
component_dropdown = widgets.Dropdown(options=['fBx', 'fBy', 'fBz', 'fEx', 'fEy', 'fEz', 'q'], value='fBz', description='Component')
timestep_slider = widgets.IntSlider(min=0, max=len(data)-2, step=1, value=0, description='Timestep')
zoom_slider = widgets.IntSlider(min=0, max=98, step=2, value=60, description='Zoom')
slice_position_slider = widgets.IntSlider(min=0, max=100, step=1, value=50, description='Slice Position')
elevation_slider = widgets.IntSlider(min=0, max=90, step=3, value=30, description='Elevation')
rotation_slider = widgets.IntSlider(min=0, max=360, step=5, value=0, description='Rotation')
dummy_checkbox = widgets.Checkbox(value=True, description='Dummy')

# Function to reset sliders
def reset_sliders(*args):
    timestep_slider.value = 0
    zoom_slider.value = 60
    slice_position_slider.value = 50
    component_dropdown.value = 'fBz'
    elevation_slider.value = 30
    rotation_slider.value = 0
    dummy_checkbox.value = True

# Create a reset button
reset_button = widgets.Button(description='Reset')
reset_button.on_click(reset_sliders)

# Create a VBox layout with all the widgets
ui = widgets.VBox([component_dropdown, timestep_slider, zoom_slider, slice_position_slider, elevation_slider, rotation_slider, dummy_checkbox, reset_button])

# Link the sliders to the plot function
out = widgets.interactive_output(plot_original, {
    'timestep': timestep_slider,
    'component': component_dropdown,
    'zoom': zoom_slider,
    'slice_position': slice_position_slider,
    'elevation': elevation_slider,
    'rotation': rotation_slider,
    'dummy': dummy_checkbox
})

# Display the widgets and output
display(ui, out)


Output()

In [53]:
data = gdftomemory("test.gdf")

#print(data[1])

max = float(0)



for i in range(len(data) - 1):
    
    q = data[i].get("d").get('q')

    mask = q != 0
    
    V1 = np.float(np.std(data[i].get("d").get('y')[mask]))
    if V1 > max:
        max = V1
        

print(max)
# print(np.std(data[0].get("d").get('y')))
# print(np.std(data[0].get("d").get('z')))


print(np.count_nonzero(data[i].get("d").get('fEx')) - len(data[i].get("d").get('fEx')))

{'time_created': '2024-08-05 13:04:40', 'creator': 'MR', 'destination': '', 'gdf_version': '1.1', 'creator_version': '1.3', 'destination_version': '0.0'}
0.005088994708282135
-103


/tmp/ipykernel_1384563/246587972.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  V1 = np.float(np.std(data[i].get("d").get('y')[mask]))
